In [ ]:
import os
import polars as pl
from kaggle_evaluation.jane_street_inference_server import JaneStreetInferenceServer

class MarketPredictionServer:
    def __init__(self):
        self.model_dev = ModelDevelopment(None)  # Initialize without data
        self.models = self.load_models()
        
    def load_models(self):
        """Load pre-trained models"""
        try:
            # Load regime-specific models
            models = {}
            model_path = "../models/"
            for regime in ['high', 'normal', 'low']:
                model_file = f"{model_path}regime_{regime}_model.joblib"
                if os.path.exists(model_file):
                    models[regime] = joblib.load(model_file)
            return models
        except Exception as e:
            print(f"Error loading models: {e}")
            return None

    def predict(self, features: Dict) -> Dict:
        """
        Main prediction endpoint for the inference server
        """
        try:
            # Convert features to DataFrame
            df = pl.DataFrame(features)
            
            # Calculate regime
            df = self.model_dev.calculate_regime(df)
            
            # Get predictions
            predictions = self.model_dev.create_ensemble_prediction(
                {'models': self.models}, df
            )
            
            # Calculate positions
            confidences = np.abs(predictions)
            positions = self.model_dev.calculate_position_sizes(
                predictions, confidences
            )
            
            # Apply risk controls
            final_positions = self.model_dev.apply_risk_controls(positions, df)
            
            return {
                'action': float(final_positions[0])  # Server expects single position
            }
            
        except Exception as e:
            print(f"Prediction error: {e}")
            return {'action': 0.0}  # Safe default